### Pre-requisites

- USE ROLE ACCOUNTADMIN;
- CREATE ROLE ML_MODEL_ROLE;
- CREATE OR REPLACE DATABASE MOVIE_RECOMMENDER_DB;
- CREATE OR REPLACE SCHEMA MOVIE_RECOMMENDER_SCHEMA;
- GRANT ALL PRIVILEGES ON DATABASE MOVIE_RECOMMENDER_DB TO ROLE ML_MODEL_ROLE;
- GRANT ALL PRIVILEGES ON SCHEMA MOVIE_RECOMMENDER_DB.MOVIE_RECOMMENDER_SCHEMA TO ROLE ML_MODEL_ROLE;
- Load all .CSVs into Tables in the SCHEMA MOVIE_RECOMMENDER_DB.MOVIE_RECOMMENDER_SCHEMA
- Create compute pool and grant privileges
- -- Create Container Runtime Notebook, import .ipynb, make sure to use ML_MODEL_ROLE

### Import all Required Libraries

In [ ]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()

### Load all required Snowflake Tables into DataFrames

In [ ]:
ratings = session.table("user_ratings").to_pandas()
ratings

In [ ]:
movie_titles = session.table("title").to_pandas()
movie_titles

In [ ]:
user_df = session.table('user').to_pandas()
user_df

### Train KNN Model

In [ ]:
user_movie_matrix = ratings.pivot(index='USER_ID', columns='ITEM_ID', values='RATING').fillna(0)
user_movie_matrix_sparse = csr_matrix(user_movie_matrix)

In [ ]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=10, n_jobs=-1)
model_knn.fit(user_movie_matrix_sparse)

### Write function to Batch Inference set of User IDs, using the trained model

In [ ]:
def recommend_movies_batch_mod(user_ids_df):
    """
    Generate movie recommendations for multiple users
    
    Parameters:
    user_ids_df (pd.DataFrame): DataFrame containing user_ids
    
    Returns:
    pd.DataFrame: DataFrame with user_ids and their movie recommendations
    """
    
    # Initialize lists to store results
    all_user_ids = []
    all_recommendations = []
    
    # Iterate through each user_id in the input DataFrame
    for user_id in user_ids_df['ID'].values:
        try:
            # Get nearest neighbors for the user
            distances, indices = model_knn.kneighbors(
                user_movie_matrix.iloc[user_id-1, :].values.reshape(1, -1), 
                n_neighbors=10
            )
            
            similar_users = indices.flatten()
            movie_scores = {}
            
            # Calculate movie scores based on similar users
            for i in similar_users:
                for movie in user_movie_matrix.columns:
                    if user_movie_matrix.iloc[user_id-1, movie-1] == 0:  # Movie not yet rated
                        movie_scores[movie] = movie_scores.get(movie, 0) + user_movie_matrix.iloc[i, movie-1]
            
            # Get top recommendations
            recommended_movies = sorted(
                movie_scores.items(), 
                key=lambda x: x[1], 
                reverse=True
            )[:3]
            
            # Format recommendations
            recommendations = []
            for movie_id, score in recommended_movies:
                #movie_name = movie_titles[movie_titles['ITEM_ID'] == movie_id]['TITLE'].values[0]
                mov_obj = {}
                mov_obj["movie_name"] = movie_titles[movie_titles['ITEM_ID'] == movie_id]['TITLE'].values[0]
                mov_obj["movie_score"] = f"{score:.2f}"
                recommendations.append(mov_obj)
            
            # Append results
            all_user_ids.append(user_id)
            all_recommendations.append(recommendations)
            
        except Exception as e:
            print(f"Error processing user_id {user_id}: {str(e)}")
            all_user_ids.append(user_id)
            all_recommendations.append("Error generating recommendations")
    
    # Create results DataFrame
    results_df = pd.DataFrame({
        'user_id': all_user_ids,
        'recommendations': all_recommendations
    })
    
    return results_df

In [ ]:
results_df = recommend_movies_batch_mod(user_df)
snp_results_df = session.create_dataframe(results_df)
snp_results_df

In [ ]:
snp_results_df.write.save_as_table(
    "movie_recommendations",
    mode="overwrite"
)

### Table saved in Snowflake with Number ID column and VARIANT recommendation column

In [ ]:
DESC TABLE MOVIE_RECOMMENDATIONS

### Use Snowflake SQL to Parse Out Recommended Movies per user

In [ ]:
SELECT 
    t."user_id",
    TRIM(PARSE_JSON(f.value):"movie_name", '"') as MOVIE_RECOMMENDATION,
    TRY_TO_NUMBER(TRIM(PARSE_JSON(f.value):"movie_score", '"')) as RECOMMENDATION_SCORE
FROM MOVIE_RECOMMENDATIONS t,
TABLE(FLATTEN(t."recommendations")) f;

### SQL Query to return only top recommended Movie per User

In [ ]:
SELECT 
    t."user_id" as user_id,
    TRIM(PARSE_JSON(f.value):"movie_name", '"') as MOVIE_RECOMMENDATION,
    TRY_TO_NUMBER(TRIM(PARSE_JSON(f.value):"movie_score", '"')) as RECOMMENDATION_SCORE
FROM MOVIE_RECOMMENDATIONS t,
TABLE(FLATTEN(t."recommendations")) f
QUALIFY ROW_NUMBER() OVER (PARTITION BY user_id ORDER BY RECOMMENDATION_SCORE desc) = 1
ORDER BY user_id asc
